In [1]:
import importlib
import time
import re
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import ChatGLM
import pandas as pd
import tqdm
import util
import uuid
from typing import Dict,List,Tuple
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter

In [2]:
poetrys = CSVLoader("由大模型处理过的文档/poetrys_72417.csv",encoding="utf-8").load()


In [8]:
new = pd.read_csv("4个数据集/poetrys_72417_noAuthor.csv")

In [5]:
spliter1 = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 64,
    length_function = len,
    is_separator_regex = True,
    separators=[u"【[\u4e00-\u9fa5]{2}】","\n\n",""]
)
spliter2 = RecursiveCharacterTextSplitter(
    chunk_size = 128,
    chunk_overlap  = 16,
    length_function = len,
    is_separator_regex = False,
)

In [10]:
res = []
for i,row in tqdm.tqdm(new.iterrows(),total=len(new)):
    head2 = ("\n标签："+row["【标签】"] if row["【标签】"] != "[]" else "标签：无") +"\n"
    head1 = row["【作者】"]+"-"+row["【朝代】"]+"："+"《"+row["【名称】"]+"》\n"
    head = head1
    res.append({"文档":head1+head2,"诗人":row["【作者】"],"朝代":row["【朝代】"]})
    contents = spliter2.split_text(row["【全文】"])
    res.extend({"文档":head+f"全文{i}："+c,"诗人":row["【作者】"],"朝代":row["【朝代】"]} 
               for i,c in enumerate(contents))
    docs = spliter1.split_text(row["【赏析】"]+"\n"+row["【翻译】"]+"\n"+row["【关于】"])
    res.extend({"文档":head+f"赏析翻译片段{i}："+d,"诗人":row["【作者】"],"朝代":row["【朝代】"]} 
               for i,d in enumerate(docs))   

100%|██████████| 72417/72417 [00:21<00:00, 3428.32it/s]


In [12]:
pd.DataFrame(res).to_csv("诗词赏析文档.csv")

In [26]:
docs = spliter1.split_documents(poetrys)

In [27]:
for i in range(10):
    print("===Sep===")
    print(docs[i].page_content)

===Sep===
【作者、朝代】: 先秦—佚名
【全文】: 关关雎鸠，在河之洲。窈窕淑女，君子好逑。
参差荇菜，左右流之。窈窕淑女，寤寐求之。
求之不得，寤寐思服。悠哉悠哉，辗转反侧。
参差荇菜，左右采之。窈窕淑女，琴瑟友之。
参差荇菜，左右芼之。窈窕淑女，钟鼓乐之。
【诗题】: 关雎
【标签】: ['古诗三百首', '初中古诗', '诗经', '爱情']
【赏析、翻译、关于】:
===Sep===
【翻译】：
译文
关关和鸣的雎鸠，相伴在河中的小洲。那美丽贤淑的女子，是君子的好配偶。
参差不齐的荇菜，从左到右去捞它。那美丽贤淑的女子，醒来睡去都想追求她。
追求却没法得到，白天黑夜便总思念她。长长的思念哟，叫人翻来覆去难睡下。
参差不齐的荇菜，从左到右去采它。那美丽贤淑的女子，奏起琴瑟来亲近她。
参差不齐的荇菜，从左到右去拔它。那美丽贤淑的女子，敲起钟鼓来取悦她。

注释
⑴关关：象声词，雌雄二鸟相互应和的叫声。雎鸠（jū jiū）：一种水鸟名，即王鴡。
⑵洲：水中的陆地。
⑶窈窕（yǎo tiǎo）淑女：贤良美好的女子。窈窕，身材体态美好的样子。窈，深邃，喻女子心灵美；窕，幽美，喻女子仪表美。淑，好，善良。
⑷好逑（hǎo qiú）：好的配偶。逑，“仇”的假借字，匹配。
⑸参差：长短不齐的样子。荇（xìng）菜：水草类植物。圆叶细茎，根生水底，叶浮在水面，可供食用。
⑹左右流之：时而向左、时而向右地择取荇菜。这里是以勉力求取荇菜，隐喻“君子”努力追求“淑女”。流，义同“求”，这里指摘取。之：指荇菜。
⑺寤寐（wù mèi）：醒和睡。指日夜。寤，醒觉。寐，入睡。又，马瑞辰《毛诗
===Sep===
淑女”。流，义同“求”，这里指摘取。之：指荇菜。
⑺寤寐（wù mèi）：醒和睡。指日夜。寤，醒觉。寐，入睡。又，马瑞辰《毛诗传笺注通释》说：“寤寐，犹梦寐。”也可通。
⑻思服：思念。服，想。 《毛传》：“服，思之也。”
⑼悠哉（yōu zāi）悠哉：意为“悠悠”，就是长。这句是说思念绵绵不断。悠，感思。见《尔雅·释诂》郭璞注。哉，语气助词。悠哉悠哉，犹言“想念呀，想念呀”。
⑽辗转反侧：翻覆不能入眠。辗，古字作展。展转，即反侧。反侧，犹翻覆。
⑾琴瑟友之：弹琴鼓瑟来亲近她。琴、瑟，皆弦乐器。琴五或七弦，瑟二十五或五十弦。友：用作动词，此处有亲近之意。这句说，用琴瑟来亲